<a href="https://colab.research.google.com/github/srijansingh53/hatErase/blob/master/LR_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
os.chdir('/content/drive/My Drive/ML Project/project')
!ls

dataset			   LR_training.ipynb   MNB.ipynb
Dataset_Exploration.ipynb  LSTM_train.ipynb    models
dataset_preprocess.ipynb   ML_Pipelines.ipynb  outputs


In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_csv('outputs/processed_wos.csv', encoding='latin-1')
df.head()

,text,label,hashtags,users,links,processed_text
0,"""like this if you are a tribe fan""",0,[],[],[],like tribe fan
1,"""you're idiot.......................""",1,[],[],[],idiot
2,"""I am a woman Babs, and the only ""war on women...",1,[],[],[],woman babs war woman see coming jackazzes like...
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1,[],[],[],wow benefitted many win year bat nice stupid
4,"""haha green me red you now loser whos winning ...",1,[],[],[],haha green red loser who winning moron


In [0]:
df['processed_text'] = df['processed_text'].str.cat(df['hashtags'], sep =' ')
df.head()

,text,label,hashtags,users,links,processed_text
0,"""like this if you are a tribe fan""",0,[],[],[],like tribe fan []
1,"""you're idiot.......................""",1,[],[],[],idiot []
2,"""I am a woman Babs, and the only ""war on women...",1,[],[],[],woman babs war woman see coming jackazzes like...
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1,[],[],[],wow benefitted many win year bat nice stupid []
4,"""haha green me red you now loser whos winning ...",1,[],[],[],haha green red loser who winning moron []


In [0]:
import re
def preprocess(text):
    text = re.sub(r'[^\w\d\s]', ' ', str(text))
    text = re.sub('[Ã]', ' ', str(text))
    return ' '.join(term for term in text.split())
df['processed_text'] = df.processed_text.apply(lambda row : preprocess(row))

In [0]:
df.head()

,text,label,hashtags,users,links,processed_text
0,"""like this if you are a tribe fan""",0,[],[],[],like tribe fan
1,"""you're idiot.......................""",1,[],[],[],idiot
2,"""I am a woman Babs, and the only ""war on women...",1,[],[],[],woman babs war woman see coming jackazzes like...
3,"""WOW & YOU BENEFITTED SO MANY WINS THIS YEAR F...",1,[],[],[],wow benefitted many win year bat nice stupid
4,"""haha green me red you now loser whos winning ...",1,[],[],[],haha green red loser who winning moron


In [0]:
pd.DataFrame(df, columns=['text','label','hashtags', 'users', 'links', 'processed_text']).to_csv('outputs/train.csv', index = False)

In [0]:
tfidf_vec = TfidfVectorizer(ngram_range=(1,3))
tfidf_text = tfidf_vec.fit_transform(df.processed_text)

In [0]:
tfidf_text.shape

(196388, 7298513)

In [0]:
tfidf_vec.idf_.shape

(7298513,)

In [0]:
import pickle
pickle.dump(tfidf_vec.idf_, open("models/idf.pickle", "wb")) # Saving idf
pickle.dump(tfidf_vec.vocabulary_, open("models/vocabulary.pickle", "wb")) # saving vocab

In [0]:
vectorizer = pickle.load(open("models/idf.pickle", "rb"))
vectorizer.shape

(7298513,)

In [0]:
vocabulary = pickle.load(open("models/vocabulary.pickle", "rb"))
len(vocabulary)

7298513

In [0]:
import joblib
joblib.dump(tfidf_vec, 'models/tfidf_vec.pkl')

['models/tfidf_vec.pkl']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_text, df['label'], test_size=0.2, random_state = 42)

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=2019, C=100, max_iter=10000).fit(X_train, y_train)

In [0]:
predictions = classifier.predict(X_test).tolist()
wrong = []
count = 0
for i in range(len(y_test)):
    if y_test.iloc[i] != predictions[i]:
        count += 1
        wrong.append(i)

      
print('Total number of test cases', len(y_test))
print('Number of wrong of predictions', count)


Total number of test cases 39278
Number of wrong of predictions 2585


In [0]:
from sklearn.metrics import classification_report
print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96     32697
           1       0.74      0.94      0.83      6581

    accuracy                           0.93     39278
   macro avg       0.86      0.93      0.89     39278
weighted avg       0.95      0.93      0.94     39278



In [0]:
import pickle
pickle.dump(classifier, open('models/LR_model.sav', 'wb'))